## Import libraries

In [2]:
import pandas as pd
import numpy as np

import tensorflow as tf

import re
from tqdm import tqdm

## Load data

In [3]:
dataset = pd.read_csv('dataset/IMDB-dataset.csv')
dataset.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


## Preprocess data

In [4]:
dataset['sentiment'] = dataset['sentiment'].map({'positive': 1, 'negative': 0})

# Ratio of positive and negative reviews
dataset['sentiment'].value_counts(normalize=True)

1    0.5
0    0.5
Name: sentiment, dtype: float64

### Split data

In [5]:
X_train_texts = dataset['review'][:49000]
y_train = dataset['sentiment'][:49000]

X_test_texts = dataset['review'][49000:]
y_test = dataset['sentiment'][49000:]

### Load embeddings

In [6]:
vocab_size = 400001
embedding_dim = 100

word_to_index = {}
index_to_word = {}
word_to_vec_map = {}
with open('embeddings/glove.6B.100d.txt', 'r', encoding='utf-8') as f:    
    for i, line in enumerate(f):
        values = line.split()
        word = values[0]
        
        coefs = np.asarray(values[1:], dtype='float32')
        
        word_to_index[word] = i
        index_to_word[i] = word
        word_to_vec_map[word] = coefs

In [7]:
word_to_index['the'], index_to_word[0], word_to_vec_map['the'].shape

(0, 'the', (100,))

#### Embedding matrix

In [8]:
# Initialize the embedding matrix with zeros
embedding_matrix = np.zeros((vocab_size, embedding_dim))

# Fill in the embedding matrix
for word, index in word_to_index.items():
    if word in word_to_vec_map:
        embedding_matrix[index] = word_to_vec_map[word]

### Tokenize

In [9]:
MAX_SENTENCE_LENGTH = 2500

def tokenize(sentences, max_sentence_length=MAX_SENTENCE_LENGTH):
    tokenized_sentences = np.zeros((len(sentences), max_sentence_length))
    
    for i, sentence in enumerate(sentences):
        words = sentence.lower().split()
        
        for j, word in enumerate(words):    
            if j == max_sentence_length:
                break
            
            # Remove punctuation from the word
            word = re.sub(r'[^\w\s]', '', word)
                        
            if word in word_to_index:
                tokenized_sentences[i, j] = word_to_index[word]
            else:
                tokenized_sentences[i, j] = 400000 # Vector for unknown words
        
    return tokenized_sentences

In [10]:
X_train_sequences = tokenize(X_train_texts)
X_test_sequences = tokenize(X_test_texts)

# Model

## Baseline
This model is a baseline for the other models. It is a simple neural network that consists of an embedding layer, a global average pooling layer, and a dense layer with sigmoid activation. **The embedding layer is trained from scratch.** Where the other models will use pre-trained embeddings, because training time would be too long.

In [10]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=MAX_SENTENCE_LENGTH),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [11]:
model.fit(X_train_sequences, y_train, epochs=20, batch_size=16, validation_data=(X_test_sequences, y_test), verbose=1)

### Results
- Small training set (1000 examples)
- Training unitl reaches max accuracy (20 to 60 epochs)
- Batch size of 16
- Adam optimizer

| Model | Accuracy | Validation Accuracy |
|-------|----------|---------------------|
| Baseline | 1.0000 | 0.8260 |
| LSTM 32 | 0.4970 | 0.5320 |
| Bi-LSTM 8 | 0.9300 | 0.7460 |
| 2 x Bi-LSTM 8 | 0.9350 | 0.7360 |
| Bi-LSTM 16 | 0.9840 | 0.7860 |
| Bi-LSTM 32 | 0.9990  | 0.7840 |
| Bi-LSTM 128 | 0.9930 | 0.7320 |
| 2 x Bi-LSTM 128 | 0.9990 | 0.7320 |

### Observations
- Simple baseline model performs surprisingly well although it's overfitting
- LSTM layers with > 32 units introduce overfitting **on a small dataset**. Training on a entire dataset should reduce overfitting. So we will try more units.
- LSTM layers with < 16 units introduce biast - can't learn function well enough
- Bidirectional LSTM layers give much better results
- Adding second layer doesn't improve performance, only increases training time
- Baseline model quickly reaches its best performance - it can't learn more complex function

### Conclusions
- Use single bidirectional LSTM layers with 16 > units
- Train on entire dataset to reduce overfitting

## Simple model

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=MAX_SENTENCE_LENGTH, weights=[embedding_matrix], trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics=['accuracy'])

In [28]:
model.fit(X_train_sequences, y_train, epochs=10, batch_size=32, validation_data=(X_test_sequences, y_test), verbose=1)

In [32]:
model = tf.keras.models.load_model('final_model.h5')

### Evaluatation

In [33]:
train_loss, train_acc = model.evaluate(X_train_sequences, y_train, verbose=0)
test_loss, test_acc = model.evaluate(X_test_sequences, y_test, verbose=0)

print(f'Train accuracy: {train_acc*100:.2f}%')
print(f'Test accuracy: {test_acc*100:.2f}%')

Train accuracy: 95.00%
Train accuracy: 90.30%


### Summary
- Final model got  **95.0%** accuracy on training set and **90.3%** accuracy on a test set.
- Great! By training on entire dataset there is only little overfitting even with 64 units.
- It's very hard to get over 90% test accuracy using this simple architecture. Units above 64 introduce overfitting. Adding more layers and layer normalization doesn't improve performance.
- Models with 64 > units can get 99% training but only 90% test accuracy.
- After 10 epochs there is only training accuracy improvement, test accuracy stays the same or even decreases.

## Attention model
### Architecture
<img src="architecture_dark.png" height="1000rem">

### Maximum sentence length
Maximum length of a sentece mush be cut from 2500 to 256 words. Otherwise, TensorFlow wouldn't be able to create the model due to exceeding recursion length.

In [11]:
MAX_SENTENCE_LENGTH = 128

units_pre = 32  # number of units for the pre-attention, bi-directional LSTM's hidden state 'a'
units_post = 64 # number of units for the post-attention LSTM's hidden state "s"

In [12]:
# Use the same layers for each attention time step
repeator = tf.keras.layers.RepeatVector(MAX_SENTENCE_LENGTH)
concatenator = tf.keras.layers.Concatenate(axis=-1)
dotor = tf.keras.layers.Dot(axes=1)

densor1 = tf.keras.layers.Dense(10, activation='tanh')
densor2 = tf.keras.layers.Dense(1, activation='relu')
activator = tf.keras.layers.Activation('softmax', name='attention_weights')

In [13]:
def one_step_attention(a, s_prev):
    """
    Calculates the context vector for a single time step.
    
    Arguments:
    ----------
    a: all hidden states of the Bi-LSTM, numpy-array of shape (batch_size, MAX_SENTENCE_LENGTH, 2*units_pre), n_a is multiplied by 2 because of the bidirectional LSTM
    s_prev: previous hidden state of the (post-attention) LSTM, numpy-array of shape (m, units_post)
    
    Returns:
    --------
    context: context vector, input of the next (post-attention) LSTM cell, numpy-array of shape (m, units_post)
    """
    # Concatenate the hidden state of the post-attention LSTM with the hidden states of the pre-attention Bi-LSTM
    s_prev = repeator(s_prev)
    concat = concatenator([a, s_prev])
    
    # Neural network to compute the attention weights
    e = densor1(concat)
    energies = densor2(e)
    alphas = activator(energies)
    
    # Compute the context vector
    context = dotor([alphas, a])
    
    return context

In [14]:
# Input to the pre-attention Bi-LSTM
inputs = tf.keras.layers.Input(shape=(MAX_SENTENCE_LENGTH,))
# Initial hidden state of the post-attention LSTM
s0 = tf.keras.layers.Input(shape=(units_post,), name='s0')
# Initial cell state of the post-attention LSTM
c0 = tf.keras.layers.Input(shape=(units_post,), name='c0')

# Set the hidden state and the cell state of the post-attention LSTM to the input states
s = s0
c = c0

# Get embeddings for the input sequence
embeddings = tf.keras.layers.Embedding(
    input_dim=vocab_size,
    output_dim=embedding_dim,
    input_length=MAX_SENTENCE_LENGTH,
    weights=[embedding_matrix],
    trainable=False
)(inputs)

# Get the hidden states of the pre-attention Bi-LSTM
a = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(units_pre, return_sequences=True))(embeddings)

# Apply dropout to the Bi-LSTM output
a = tf.keras.layers.Dropout(0.2)(a)

# Create post-attention LSTM cell
post_activation_LSTM = tf.keras.layers.LSTM(units_post, return_state=True)

# Iterate over the time steps
for t in tqdm(range(MAX_SENTENCE_LENGTH)):
    # Get the context vector for time step t
    context = one_step_attention(a, s)
    
    # Feed the context vector to the post-attention LSTM cell
    s, _, c = post_activation_LSTM(context, initial_state=[s, c])
    
output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(s)

# Add dropout to the final state
s = tf.keras.layers.Dropout(0.2)(s)

model = tf.keras.Model(inputs=[inputs,s0,c0], outputs=output_layer)

100%|██████████| 128/128 [00:29<00:00,  4.40it/s]


In [16]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 128, 100)     40000100    ['input_1[0][0]']                
                                                                                                  
 bidirectional (Bidirectional)  (None, 128, 64)      34048       ['embedding[0][0]']              
                                                                                                  
 s0 (InputLayer)                [(None, 64)]         0           []                               
                                                                                              

### Preprocessing
Dataset must be preprocessed using 128 as the maximum length of a sentence. Otherwise, the model wouldn't be able to be created due to exceeding recursion length.

In [18]:
X_train_sequences = tokenize(X_train_texts, MAX_SENTENCE_LENGTH)
X_test_sequences = tokenize(X_test_texts, MAX_SENTENCE_LENGTH)

### Training

In [19]:
s0 = np.zeros((X_train_sequences.shape[0], units_post))
c0 = np.zeros((X_train_sequences.shape[0], units_post))

s0_test = np.zeros((X_test_sequences.shape[0], units_post))
c0_test = np.zeros((X_test_sequences.shape[0], units_post))

model.fit([X_train_sequences, s0, c0], y_train, epochs=30, batch_size=32, validation_data=([X_test_sequences, s0_test, c0_test], y_test), verbose=1)

Epoch 1/30


In [ ]:
model.save('lstm_with_attention_dropout.h5')

### Prediction

In [35]:
review = [
    "The movie was great. I enjoyed every second of it although actors play wasn't perfect but plot made up for it. I would recommend it to others.",
    "I loved this movie. It was so funny and entertaining.",
    "This movie was terrible. The plot was boring and the acting was awful.",
    "Funny movie but not an amazing one. Best to watch with your friends.",
    "Not a good movie. I wasn't really satisfied with the actors play. I wouldn't recommend it to others.",
    "A good movie. I was satisfied with the actors play. I would recommend it to others."
]

review_sequence = tokenize(review)

prediction = model.predict(review_sequence).flatten() > 0.5

# Map True to Positive and False to Negative
np.where(prediction, "Positive", "Negative").tolist()

1/1 [==============================] - 0s 111ms/step


['Positive', 'Positive', 'Negative', 'Positive', 'Negative', 'Positive']